In [1]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
print('Libraries installed!')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0:00:00  42.29 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.44 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  37.23 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize
import json
import requests
from bs4 import BeautifulSoup
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

## Web scraping

In [3]:
# open web page containing necessary data with Beautiful Soup

location_name=[]
location_price=[]

for page in range(1,95):
    data = requests.get('https://www.makaan.com/price-trends/property-rates-for-buy-in-mumbai?page={}'.format(page)).text
    soup = BeautifulSoup(data, 'html.parser')
    for row in soup.find('table').find_all('tr'): 
        cells = row.find_all('td')
        if(len(cells) > 0):
            location_name.append(cells[0].text)
            location_price.append(cells[2].text)
            
print('Finished collecting data!')

Finished collecting data!


In [4]:
for n, price in enumerate(location_price):
    
    total=''
    
    for i in re.findall(r'\d+(?:\.\d+)?', price):
        total+=i
        location_price[n]=float(total)
        
location_price

[99566.03,
 83951.24,
 64145.5,
 58929.63,
 34268.08,
 141962.39,
 35898.06,
 28654.72,
 42118.27,
 35204.85,
 39276.99,
 99751.54,
 66929.58,
 82723.0,
 10903.97,
 27325.0,
 27917.84,
 17677.86,
 63886.17,
 23831.61,
 84475.69,
 9718.55,
 49422.17,
 111271.41,
 34446.95,
 51742.13,
 31929.96,
 38154.21,
 67658.29,
 17543.16,
 58572.43,
 3591.3,
 316.72,
 7432.88,
 31257.87,
 22728.23,
 25422.81,
 24365.35,
 57311.11,
 67264.31,
 32519.35,
 66827.6,
 68115.26,
 17837.09,
 47444.65,
 1392.21,
 63233.94,
 42585.06,
 69366.81,
 16908.49,
 86570.95,
 79759.73,
 18139.29,
 54307.39,
 76433.63,
 62317.6,
 61921.7,
 77173.58,
 85937.21,
 8566.04,
 17831.35,
 32704.43,
 3257.58,
 37996.33,
 13419.2,
 54571.4,
 34621.57,
 12130.35,
 6473.2,
 8362.77,
 20420.71,
 69942.88,
 103858.97,
 108446.21,
 37905.44,
 45974.48,
 36217.03,
 56339.03,
 98921.37,
 27328.34,
 18466.55,
 26932.57,
 30490.5,
 45309.69,
 4784.2,
 10445.78,
 5985.38,
 28264.67,
 26416.36,
 32580.92,
 12773.45,
 5689.33,
 68785.2,

In [5]:
location_name

['Thane West',
 'Kharghar',
 'Mira Road East',
 'Kandivali East',
 'Goregaon East',
 'Powai',
 'Dombivali',
 'Neral',
 'Titwala',
 'Chembur',
 'Ulwe',
 'Bhandup West',
 'Seawoods',
 'Bandra West',
 'Virar',
 'Jogeshwari East',
 'Malad West',
 'Kamothe',
 'Andheri West',
 'Panvel',
 'Malad East',
 'Badlapur East',
 'Mulund West',
 'Nerul',
 'Dombivali East',
 'Borivali East',
 'Kandivali West',
 'Andheri East',
 'Goregaon West',
 'Taloja',
 'Chandivali',
 'Chembur East',
 'Ambernath East',
 'Nala Sopara',
 'Borivali West',
 'Tilak Nagar',
 'Ghansoli',
 'Badlapur West',
 'Kolshet Road',
 'Manpada',
 'Kanjurmarg',
 'Parel',
 'Khar West',
 'Kalyan West',
 'Majiwada',
 'Karanjade',
 'Belapur',
 'Ghodbunder Road',
 'Lower Parel',
 'Koperkhairane',
 'Navi Basti',
 'Sanpada',
 'Virar West',
 'Mulund',
 'Worli',
 'Andheri',
 'Santacruz West',
 'West Amardeep Colony',
 'Prabhadevi',
 'Ambernath West',
 'Airoli',
 'Badlapur',
 'Jogeshwari West',
 'Dahisar',
 'Bhiwandi',
 'Wadala',
 'Bhayandar Eas

In [6]:
df = pd.DataFrame({'Place':location_name, 'PricePersqFt':location_price})
df.head()

,Place,PricePersqFt
0,Thane West,99566
1,Kharghar,83951.2
2,Mira Road East,64145.5
3,Kandivali East,58929.6
4,Goregaon East,34268.1


In [7]:
df.count()

Place           5635
PricePersqFt    5635
dtype: int64

In [9]:
df[df['PricePersqFt']=='-'].count()

Place           4097
PricePersqFt    4097
dtype: int64

In [10]:
df_mumbai=df[df['PricePersqFt']!='-'].reset_index(drop=True)

In [11]:
lat=[]
long=[]
for row in df_mumbai.itertuples():
    address = '{},Mumbai'.format(row.PricePersqFt)
    try:
        geolocator = Nominatim(user_agent="tl-toronto-neigh")
        location = geolocator.geocode(address, timeout=10)
        lat.append(location.latitude)
        long.append(location.longitude)
    except:
        lat.append(np.nan)
        long.append(np.nan)

print('All Done')

All Done


In [12]:
df_mumbai['latitude']=lat
df_mumbai['longitude']=long

In [13]:
df_mumbai.dropna(inplace=True, axis='index').reset_index(drop=True)

In [14]:
df_mumbai.count()

Place           313
PricePersqFt    313
latitude        313
longitude       313
dtype: int64

# Explore and cluster the neighborhoods in Toronto

In [16]:
mumbai_lat=19.0760
mumbai_long=72.8777

#Using Folium to create a Map of Toronto with Boroughs markers on top

map_mumbai = folium.Map(location=[mumbai_lat, mumbai_long], zoom_start=11)

for lat, long, place in zip(df_mumbai['latitude'], df_mumbai['longitude'], df_mumbai['Place']):
    label = "{}".format(place)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)
    
map_mumbai

**Inorder to reduce the numbers of calls to FourSquare API, we will only explore only random 40 places from the list.**

In [21]:
mumbai_explore=df_mumbai.sample(40).reset_index(drop=True)
mumbai_explore.head()

**Using FourSquare API**

In [26]:
radius = 500
LIMIT = 100

venues = []

for lat, long, place in zip(mumbai_explore['latitude'], mumbai_explore['longitude'], mumbai_explore['Place']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=CNBMOFBQCRK4JMLSE2GNEMVTK4VJFOSKWC10FUMKABQMF10F&client_secret=X13URYICDGIRILRI5SOPNTBUW2GMSLIDCZ5MVYYVW0YTGW3P&v=20180605&ll={},{}&radius={}&limit={}".format(
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            place,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [27]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Place', 'PlaceLatitude', 'PlaceLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

In [29]:
venues_df

,Place,PlaceLatitude,PlaceLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,New Mhada Colony,19.063003,72.922195,Alviya Telecom,19.065168,72.923334,Shop & Service
1,New Mhada Colony,19.063003,72.922195,cafe faisal,19.060947,72.920446,Indian Restaurant
2,New Mhada Colony,19.063003,72.922195,Lazeez Sheekh,19.063987,72.919090,Fried Chicken Joint
3,New Mhada Colony,19.063003,72.922195,shivaji nagar bus stop,19.062082,72.918625,Bus Station
4,New Mhada Colony,19.063003,72.922195,Shree Chamunda Super store,19.059062,72.923993,Department Store
5,New Mhada Colony,19.063003,72.922195,Rehmani,19.066390,72.919096,Restaurant
6,Shilphata Diva Naka,19.043255,72.824899,Bandra Sea Face,19.044217,72.820290,Beach
7,Shilphata Diva Naka,19.043255,72.824899,Mafco Farm Fair,19.039117,72.823233,Food Truck
8,Shilphata Diva Naka,19.043255,72.824899,Recla,19.043274,72.824721,Racetrack
9,Shilphata Diva Naka,19.043255,72.824899,AB Celestial,19.041031,72.823029,Boat or Ferry


**Check how many venues returned for each place**

In [30]:
venues_df.groupby('Place')['VenueName'].count()

Place
Ashok Chakravarti Road             20
Azad Nagar Lane 3                  20
Bhakti Park Wadala East             4
Bhawani Sankar                     20
Bhiwandi                            4
Borivali West Old MHB Colony       20
Chandansar                        100
Dhokali Naka                       20
Diva Gaon                         100
Gandhigram Road                    10
Housing Board Colony                5
Kanchpada                          95
Kolshet Thane                      20
L I C Colony                       20
Lodha Luxuria                      20
Lokhandwala kandivali              20
Mahavir Nagar                      20
Model Jp Road                      20
Mulund East Gavanpada              20
Mumbai Agra National Highway       20
Mumbai Pandharpur Road              4
Nadiadwala Colony                   4
New Golden Nest                    20
New Mhada Colony                    6
Palava City                         4
Ranjanpada                          4
Sector

In [31]:
venues_df['VenueCategory'].nunique()

96

**Analyze venues in each area**

In [43]:
venues_onehot=pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
venues_onehot['Place']=mumbai_explore['Place']
# move place column to the first column
fixed_columns = list(venues_onehot.columns[-1:]) + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
venues_onehot.head()

,Place,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Beach,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bridge,Bus Station,Café,Caucasian Restaurant,Chinese Restaurant,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food Truck,French Restaurant,Fried Chicken Joint,Greek Restaurant,Harbor / Marina,Historic Site,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irani Cafe,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Lounge,Lyonese Bouchon,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music School,Music Venue,Neighborhood,Nightclub,Opera House,Optical Shop,Other Repair Shop,Palace,Pedestrian Plaza,Pizza Place,Platform,Plaza,Portuguese Restaurant,Pub,Public Art,Racetrack,Restaurant,Roller Rink,Russian Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shop & Service,Soup Place,Spa,Steakhouse,Strip Club,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Bar
0,New Mhada Colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Prem Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Shilphata Diva Naka,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Umroli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wada,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Get 10 most occurance venue types in each area**

In [87]:
venues_freq = venues_onehot.groupby('Place').mean().reset_index()
order_prefix=['st','nd','rd']
col=[]

for i in np.arange(10):
    try:
        col.append('{}{} Common Venue'.format(i+1, order_prefix[i]))
    except:
        col.append('{}th common venue'.format(i+1))
place_venue_common=pd.DataFrame(columns=(['Place']+col))  
place_venue_common['Place']=venues_freq['Place']
for i in np.arange(venues_freq.shape[0]):
    categories=venues_freq.iloc[i, 1:]
    categories=categories.sort_values()
    place_venue_common.iloc[i, 1:]=categories.index.values[0:10]
place_venue_common

,Place,1st Common Venue,2nd Common Venue,3rd Common Venue,4th common venue,5th common venue,6th common venue,7th common venue,8th common venue,9th common venue,10th common venue
0,Ashok Chakravarti Road,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue
1,Azad Nagar Lane 3,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue
2,Bhakti Park Wadala East,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue
3,Bhawani Sankar,Arcade,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue,Music School
4,Bhiwandi,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue
5,Borivali West Old MHB Colony,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue
6,Chandansar,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue
7,Dhokali Naka,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue
8,Diva Gaon,Arcade,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue,Music School
9,Gandhigram Road,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue


**Clustering areas**

In [89]:
kclusters = 3
venues_freq_clustering = venues_freq.drop(['Place'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_freq_clustering)

clustered_df = mumbai_explore
clustered_df['Cluster'] = kmeans.labels_

clustered_df = clustered_df.join(place_venue_common.set_index('Place'), on='Place')
clustered_df.sort_values(['Cluster'] + col, inplace=True)
clustered_df

,Place,PricePersqFt,latitude,longitude,Cluster,1st Common Venue,2nd Common Venue,3rd Common Venue,4th common venue,5th common venue,6th common venue,7th common venue,8th common venue,9th common venue,10th common venue
0,New Mhada Colony,14583.3,19.063003,72.922195,0,Arcade,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue,Music School
5,Lodha Luxuria,2110.18,18.938771,72.835335,0,Arcade,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue,Music School
8,Diva Gaon,5224.78,59.932069,30.312396,0,Arcade,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue,Music School
11,Kanchpada,22343.7,45.768123,4.832960,0,Arcade,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue,Music School
16,charkop sector 9,13392.5,19.043809,72.825955,0,Arcade,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue,Music School
27,Ranjanpada,8753.6,19.043340,72.824851,0,Arcade,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue,Music School
36,Bhawani Sankar,28125,18.938771,72.835335,0,Arcade,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue,Music School
1,Prem Nagar,18269.4,19.061910,72.928127,0,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue
3,Umroli,463.7,18.938771,72.835335,0,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue
4,Wada,7698.17,18.938771,72.835335,0,Arcade,Pizza Place,Pedestrian Plaza,Palace,Other Repair Shop,Optical Shop,Opera House,Nightclub,Neighborhood,Music Venue


**Show those clusters onto a map**

In [91]:
# create map
map_clusters = folium.Map(location=[mumbai_lat, mumbai_long], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, place, cluster in zip(clustered_df['latitude'], clustered_df['longitude'], clustered_df['Place'], clustered_df['Cluster']):
    label = folium.Popup('{}  - Cluster {}'.format(place, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Upon observing the result, we can name the clusters as follow:

<br>Cluster 0: Living area (with mostly park, trail, school; and some small businesses)</br>
<br>Cluster 1: Roselawn - Central Toronto (nothing here except a garden)</br>
<br>Cluster 2: Business area (with lots of business venues)</br>